In [17]:
from pathlib import Path

# Automatically get the base path of your project
base_path = Path.cwd().parents[0]  # adjust .parents[0] if needed
print("Base path of the project:", base_path)

Base path of the project: c:\Users\Yusuf\OneDrive\LST\Derde_jaar\Y3Q4\Metaproteomics_with_db


## Step 1: Define the Query Database and Label

In this step, we specify which protein database we want to compare against the **metagenomics reference database**. This custom query database contains proteins we previously selected, clustered, or filtered using specific strategies (e.g., top-ranked taxa, clustering at 95% identity, etc.).

### Why we do this:
- It defines the **input protein database** for the overlap comparison.
- This helps assess how well your constructed protein database represents the proteomic space found in the metagenomics sample.
- The `query_db_label` ensures all results and logs are clearly labeled, especially when analyzing multiple databases.

### How this fits in the pipeline:
- It sets the stage for all subsequent steps, including DIAMOND alignment and overlap analysis.
- Every result calculated later (e.g., matched proteins, percentages) will be based on this database.


In [2]:
# Paste the file path of the database you want to compare with the metagenomics database between the quotes
query_db = r"C:\Users\Yusuf\OneDrive\LST\Derde_jaar\Y3Q4\Metaproteomics_with_db\pipelines\Diamond_alignments\session_diamond_align_2025-06-01_21-39-27\all_diamond_proteins_combined_uniprotkb.fasta"

# Define a short label for the database; this will be used in print statements and summary reports
query_db_label = "Diamond_proteins_uniprotkb"

In [27]:
def count_fasta_entries(fasta_path):
    count = 0
    with open(fasta_path, 'r') as f:
        for line in f:
            if line.startswith('>'):
                count += 1
    return count

# Assuming query_db is defined
num_proteins = count_fasta_entries(query_db)
print(f"Total number of protein entries in {query_db}: {num_proteins}")


Total number of protein entries in C:\Users\Yusuf\OneDrive\LST\Derde_jaar\Y3Q4\Metaproteomics_with_db\pipelines\Diamond_alignments\session_diamond_align_2025-06-01_21-39-27\all_diamond_proteins_combined_uniprotkb.fasta: 1246240


## Step 2: Build a DIAMOND Database from the Query FASTA

DIAMOND requires that input protein sequences be compiled into its own binary `.dmnd` format for fast and efficient alignments. This step ensures that your custom protein database is in the correct format before running any alignments.

### Why we do this:
- DIAMOND does not work directly with `.fasta` files — it needs a `.dmnd` file.
- By automating this step, we avoid errors and ensure the database is ready for alignment.
- The check prevents unnecessary recomputation if the `.dmnd` already exists.

### How this fits in the pipeline:
- This is a one-time setup step per protein database.
- It prepares your custom FASTA file for alignment against the metagenomics database in the next step.


In [24]:
import os
import subprocess
from pathlib import Path

# === Configuration ===

# Get the base directory of the project (1 level above current working directory)
base_path = Path.cwd().parents[0]

# Path to the DIAMOND executable (adjust if needed)
diamond_exe = base_path / "software_tools" / "diamond.exe"

# Derive the path to the DIAMOND database (.dmnd) from the query FASTA file path
# Example: input.fasta → input.dmnd
diamond_db_path = os.path.splitext(query_db)[0]


# === Function to check and create DIAMOND database ===

def ensure_diamond_db(fasta_file: str, diamond_db: str):
    # Only build the .dmnd file if it does not already exist
    if not os.path.exists(diamond_db + ".dmnd"):
        print(f"Creating DIAMOND database for: {fasta_file}")
        
        # Run DIAMOND makedb command to create a .dmnd from the FASTA input
        subprocess.run([
            diamond_exe, "makedb",
            "--in", fasta_file,    # Input FASTA file
            "-d", diamond_db       # Output DB name (without .dmnd extension)
        ], check=True)

        print("DIAMOND DB created.")
    else:
        # If the .dmnd file already exists, skip rebuilding
        print(f"DIAMOND DB already exists: {diamond_db}.dmnd")

# === Call the function to ensure DIAMOND DB exists for the query FASTA ===
ensure_diamond_db(query_db, diamond_db_path)


Creating DIAMOND database for: C:\Users\Yusuf\OneDrive\LST\Derde_jaar\Y3Q4\Metaproteomics_with_db\pipelines\Diamond_alignments\session_diamond_align_2025-06-01_21-39-27\all_diamond_proteins_combined_uniprotkb.fasta
DIAMOND DB created.


## Step 3: Run DIAMOND Alignment Against Metagenomics Proteins

This step performs a **DIAMOND alignment** to compare your custom protein database against the metagenomics reference database. The goal is to find which metagenomics proteins are captured (i.e. matched) by your query database using meaningful alignment thresholds.

### Why we do this:
- This allows you to **quantify overlap** between your reduced database and the full metagenomics database.
- DIAMOND is used here for its **speed and accuracy** in aligning large protein sets.
- A progress bar is added for usability, especially when aligning millions of sequences.

### Alignment parameters:
- `--id 70`: minimum 70% identity — moderate stringency to ensure biologically relevant matches.
- `--query-cover 80`: at least 80% of the query must align.
- `--evalue 1e-5`: filters out random or weak hits.
- `--max-target-seqs 1`: only retain the best match per query.

### How this fits in the pipeline:
- This step produces the `.m8` alignment file used for downstream **coverage and overlap statistics**.
- It forms the basis for evaluating the **representativeness** of your custom database.


In [4]:
import subprocess
import os
from pathlib import Path
from tqdm import tqdm

# === Paths ===

# Get base project directory
base_path = Path.cwd().parents[0]

# Path to DIAMOND executable
diamond_exe = base_path / "software_tools" / "diamond.exe"

# Path to the metagenomics reference protein database (FASTA)
metagen_db = base_path / "db_psm_results" / "Metagenomics_db" / "GW_proteins_CD_clean.fasta"

# Output path for the alignment results (.m8 format)
output_file = base_path / "db_results_analysis" / "Diamond_alignments" / f"{query_db_label}_aligned_with_metagenomicsdb.m8"


# === Step 1: Estimate number of proteins in the metagenomics DB ===

def count_fasta_headers(fasta_path):
    # Counts the number of FASTA headers (lines starting with '>') in the file
    with open(fasta_path) as f:
        return sum(1 for line in f if line.startswith(">"))

# Count proteins to configure progress bar
n_proteins = count_fasta_headers(metagen_db)
print(f"Estimating {n_proteins} proteins for alignment...")


# === Step 2: Run DIAMOND alignment with a live progress bar ===

print("Running DIAMOND alignment...")

# Set up a progress bar using tqdm
with tqdm(total=n_proteins, desc="Aligning proteins", unit="seq") as pbar:
    
    # Launch DIAMOND as a subprocess
    process = subprocess.Popen([
        str(diamond_exe), "blastp",              # DIAMOND mode: protein vs protein
        "--query", str(metagen_db),              # Use the metagenomics DB as query
        "--db", str(diamond_db_path) + ".dmnd",  # Search against the prebuilt DIAMOND DB
        "--out", str(output_file),               # Save alignment output here
        "--outfmt", "6",                         # BLAST tabular output (.m8)
        "--threads", "12",                       # Use 12 CPU threads (adjustable)
        "--max-target-seqs", "1",                # Only return top 1 best match per query
        #"--evalue", "1e-5",                      # E-value cutoff to avoid random hits
        #"--id", "60",                            # Minimum 60% identity required
        #"--query-cover", "70",                   # Minimum 70% of query must align
        "--fast"
    ], stderr=subprocess.PIPE, text=True)

    # Read DIAMOND's stderr to update progress bar (if supported by output)
    while True:
        line = process.stderr.readline()
        if not line:
            break
        if "Processed" in line:
            try:
                # Try to extract processed sequence count and update progress bar
                processed = int(line.strip().split()[1])
                pbar.n = processed
                pbar.refresh()
            except:
                pass

    # Wait for DIAMOND to finish
    process.wait()

    # Final update of progress bar after completion
    pbar.n = n_proteins
    pbar.refresh()

# Notify user that alignment has completed
print(f"Alignment complete. Output saved to: {output_file}")


Estimating 1647471 proteins for alignment...
Running DIAMOND alignment...


Aligning proteins: 100%|██████████| 1647471/1647471 [09:35<00:00, 2860.21seq/s]

Alignment complete. Output saved to: c:\Users\Yusuf\OneDrive\LST\Derde_jaar\Y3Q4\Metaproteomics_with_db\db_results_analysis\Diamond_alignments\Diamond_proteins_uniprotkb_aligned_with_metagenomicsdb.m8


## Step 4: Analyze Alignment Results — Overlap and Coverage Statistics

After the DIAMOND alignment, we now compute how much **overlap** exists between the proteins in the metagenomics database and your custom query database.

### Why we do this:
- To evaluate **how well your constructed protein DB covers the sample's proteomic space**.
- It allows you to determine:
  - Which proteins in your custom DB were actually found in the metagenomics dataset.
  - How many proteins from the metagenomics dataset were captured by your DB.

### Metrics calculated:
- Total proteins in each database.
- Number of **matched** and **unmatched** proteins.
- Percentage of proteins in the query DB found in the metagenomics DB (forward overlap).
- Percentage of proteins in the metagenomics DB matched by the query DB (reverse coverage).

### How this fits in the pipeline:
- These statistics give **quantitative feedback** about how representative and useful your custom database is for peptide spectrum matching.


In [42]:
from pathlib import Path

# === Paths ===
alignment_result = output_file

# === Load all protein IDs from FASTA ===

def load_all_fasta_headers(fasta_path):
    """
    Load all protein IDs from a FASTA file (one per header), including duplicates.
    Extract only the first token after '>'.
    """
    headers = []
    with open(fasta_path) as f:
        for line in f:
            if line.startswith(">"):
                header = line[1:].strip().split()[0]
                if '|' in header:
                    parts = header.split('|')
                    headers.append(parts[1])  # UniProt accession (e.g. A0A697HT45)
                else:
                    headers.append(header)
    return headers

# Load all headers
querydb_ids = load_all_fasta_headers(query_db)  # Now includes all 1.24M proteins
meta_ids = set(load_all_fasta_headers(metagen_db))  # still deduplicated

# === Parse alignment results ===

aligned_querydb_ids = set()
aligned_meta_ids = set()

with open(alignment_result) as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) >= 2:
            query_id = parts[0]
            subject_id = parts[1]
            # Clean up subject and query IDs just like above
            query_id = query_id.split('|')[1] if '|' in query_id else query_id
            subject_id = subject_id.split('|')[1] if '|' in subject_id else subject_id
            aligned_meta_ids.add(query_id)
            aligned_querydb_ids.add(subject_id)

# === Count matches ===
querydb_total = len(querydb_ids)
querydb_matched = sum(1 for pid in querydb_ids if pid in aligned_querydb_ids)
querydb_unmatched = querydb_total - querydb_matched

# === Report ===
print(f"Total proteins in {query_db_label} DB: {querydb_total}")
print(f"{query_db_label} proteins found in metagenomics DB: {querydb_matched}")
print(f"{query_db_label} proteins NOT found in metagenomics DB: {querydb_unmatched}")
print(f"Percentage of {query_db_label} proteins found in metagenomics DB: {querydb_matched / querydb_total * 100:.2f}%")

print(f"\nTotal proteins in metagenomics DB: {len(meta_ids)}")
print(f"Metagenomics proteins that match {query_db_label} DB: {len(aligned_meta_ids)}")
print(f"Metagenomics proteins NOT matching {query_db_label} DB: {len(meta_ids - aligned_meta_ids)}")
print(f"Percentage of metagenomics proteins matching {query_db_label} DB: {len(aligned_meta_ids) / len(meta_ids) * 100:.2f}%")


Total proteins in Diamond_proteins_uniprotkb DB: 1246240
Diamond_proteins_uniprotkb proteins found in metagenomics DB: 431383
Diamond_proteins_uniprotkb proteins NOT found in metagenomics DB: 814857
Percentage of Diamond_proteins_uniprotkb proteins found in metagenomics DB: 34.61%

Total proteins in metagenomics DB: 1647471
Metagenomics proteins that match Diamond_proteins_uniprotkb DB: 388367
Metagenomics proteins NOT matching Diamond_proteins_uniprotkb DB: 1259104
Percentage of metagenomics proteins matching Diamond_proteins_uniprotkb DB: 23.57%
